In [ ]:
import h5py
path = "/Volumes/HEIU/data/bulkice_train.hdf5"
print(path)
f = h5py.File(path, "r")


In [ ]:
from dataset import AFMDataset
path = "/Volumes/HEIU/data/bulkice_train.hdf5"

dts = AFMDataset(path)
fn, afm, label_type, label_pos = dts[0]

In [ ]:
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
plt.imshow(make_grid(afm.transpose(1,0)).permute(1,2,0))

In [ ]:
f['T180_13015'].afm

In [ ]:
from dataset.dataset import z_sampler
z_sampler(10, 4, True)

In [ ]:
import torch
from torch import nn
import torchvision
import matplotlib.pyplot as plt
import time
from dataset import transform as tf

pics = torch.ones(1, 8, 100, 100) * 0.9
pics[..., 50:, 0:50] = 0.1
pics[..., :50, 50:] = 0.1

transform = nn.Sequential(
    tf.PixelShift(),
    tf.Cutout(),
    tf.ColorJitter(),
    tf.Noisy(),
    tf.Blur(),
)
t = time.time()
for i in range(1):
    pics = transform(pics)

print(time.time() - t)


T = torchvision.utils.make_grid(pics.transpose(1,0),pad_value = 0.5)
plt.imshow(T.permute(1,2,0))


In [ ]:
import torch
torch.arange(10)

In [ ]:
a = torch.ones(100, 100)
a[10:20, 10:20]

In [ ]:
import torch
a = torch.rand(100, 100, 3)
b = torch.rand(100, 100) > 0.5
a[b]

In [ ]:
import torch
a = torch.ones((50, 50, 5))
c = torch.rand((50, 50, 5, 3))
mask = a.nonzero()
print(mask.T.shape)
a[tuple(mask.T)]
(c[tuple(mask.T)] + mask) / (50, 50, 5)

In [ ]:
import torch
from utils import poscar
box_cls = torch.randint(0, 3, (3, 50, 50), dtype=torch.long)
box_off = torch.rand((3, 50, 50, 3))
print(poscar.boxncls2pos_torch(box_cls, box_off))
print(poscar.boxncls2pos_np(box_cls.numpy(), box_off.numpy()))

In [ ]:
import torch
from utils.metrics import Analyser
from utils import poscar
from dataset import AFMDataset
path = "/Volumes/HEIU/data/bulkice_train.hdf5"

dts = AFMDataset(path)
fn, afm, label_type, label_pos = dts[0]
pred_types, pred_pos = poscar.targ2pred(label_type, label_pos)
a = torch.jit.script(Analyser())
pred_types = pred_types.unsqueeze(0).repeat((10, 1, 1, 1, 1))
pred_pos  = pred_pos.unsqueeze(0).repeat((10, 1, 1, 1, 1))
label_type = label_type.unsqueeze(0).repeat((10, 1, 1, 1))
label_pos = label_pos.unsqueeze(0).repeat((10, 1, 1, 1, 1))

In [ ]:
import time
t = time.time()
for i in range(1000):
    a(pred_types, pred_pos, label_type, label_pos)
print(time.time() - t)

In [ ]:
a = torch.rand((100, 100, 3))
mask = torch.nonzero(a, as_tuple=True)
a[mask[0], mask[1], mask[2]]

In [ ]:
import time
import torch
from utils.metrics import ConfusionMatrixCounter
cm = torch.randint(0, 100, (1, 2, 1, 3)).numpy()
t = time.time()
ConfusionMatrixCounter._count(cm)

In [ ]:
import torch
from model import unet_onehot
m = unet_onehot()
img = torch.rand((1, 1, 10, 100, 100))
m(img)

In [ ]:
import torch
import numpy as np
from utils.metrics import Analyser
from utils import poscar
from dataset import AFMDataset
path = "/Volumes/HEIU/data/bulkice_train.hdf5"

dts = AFMDataset(path)
fn, afm, label_type, label_pos = dts[0]
print(fn)
typ, final, _ = poscar.boxncls2pos_torch(label_type, label_pos)
typ = typ.numpy()
final = final.numpy()
print(len(final))
pos2 = np.concatenate([final[typ == i] for i in range(1,3)])
poscar.save("test.poscar", [3.0, 25.0, 25.0], ['O', 'H'], [48, 96], pos2)

In [ ]:
import numpy as np
a = ["1", "2", "3"]
np.array(a, dtype=np.float32)

In [ ]:
int("9246.0")

In [ ]:
import numpy as np
from numba import jit




# @jit(nopython=True)
def getWaterRotate(tag: np.ndarray, inv_ref: np.ndarray) -> np.ndarray:
    O, H1, H2 = tag
    H1, H2 = H1 - O, H2 - O
    H1 = H1
    H2 = H2
    H3 = np.cross(H1, H2)
    if H3[2] < 0:
        tag[0] = H2
        tag[1] = H1
        tag[2] = -H3
    else:
        tag[0] = H1
        tag[1] = H2
        tag[2] = H3
    return inv_ref @ tag

ang = 104.52 / 180 * np.pi
# v = np.array([0, 0, 1]) * 0.9572
# u = np.array([np.sin(ang), 0, np.cos(ang)]) * 0.9572
# r = np.cross(v, u)
# print(v, u,r)
initpos = np.array([
        [ 0.         , 0.         , 0.        ],
        [ 0.         , 0.         , 0.9572    ],
        [ 0.9266272  , 0.         ,-0.23998721],
    ])

initdir = np.array([
        [ 0.         , 0.         , 0.9572    ],
        [ 0.9266272  , 0.         ,-0.23998721],
        [ 0.         , 0.88696756 , 0.        ]
    ])



pos = np.asarray([[
        [6.20233009,  6.80328286,  0.71540000], 
        [6.72382009,  6.43174286,  0.00390000], 
        [5.46304009,  6.20268286,  0.81010000]
        ],
        [
        [14.10287009, 18.97821286,  0.47590000], 
        [14.35087009, 19.79591286,  0.04450000], 
        [13.24817009, 19.16231286,  0.86570000]
        ]])

def getRotate(final: np.ndarray, init: np.ndarray) -> np.ndarray:
    """
    Rotate init to final, init @ R = final

    Args:
        final (np.ndarray): N 3 3
        init (np.ndarray): 3 3

    Returns:
        np.ndarray: N 3 3
    """
    init = np.linalg.inv(init)
    final = final - final[:, (0,), :]
    cross = np.cross(final[:,1], final[:, 2])
    
    final = np.stack([final[:,1], final[:, 2], cross], axis=1)
    R = np.einsum("ij, Njk -> Nik", init, final)
    # print(np.linalg.det(R))
    return R
    
R = getRotate(pos, initdir)

R1 = R[0]
print(initpos @ R1)
# print(np.einsum("Nij, kj -> Nki", R, initpos))

import time
t = time.time()
R2 = getWaterRotate(pos[0].copy(), np.linalg.inv(initdir))
print(time.time() - t)

print(initpos @ R2[(0,2,1),:])
print(initpos @ R2)
# print(pos[0] - pos[0,(0,), :])

In [ ]:
print(np.cross(R1[0], R1[1]) - R1[2])
print(np.cross(R2[0], R2[1]) - R2[2])
print(R1)
print(R2)
print(R2[(0,-2,-1),:])

In [ ]:
import torch
from model import VAEunet
import utils
net = VAEunet(channel_mult = (1, 2, 3, 4))
temp = torch.rand((1, 1))
print("\n".join(utils.model.model_structure(net)))
image = torch.rand((1, 10, 4,25, 25))
mu, nu, x = net(image, temp)

In [ ]:
from dataset import AFMGenDataset
dts = AFMGenDataset("/Volumes/HEIU/data/ice_8_4A/ice_8_4A_train.hdf5")
name, box4a, box8a = dts[0]

In [ ]:
print(len(box4a[...,0].nonzero()))
print(len(box8a[...,0].nonzero()))

In [ ]:
import torch
a = torch.zeros(10)
a[(0,1,2),] = 1
print(a)

In [ ]:
def argmatch(pred: Tensor, targ: Tensor, cutoff: float) -> tuple[Tensor]:
    dis = torch.cdist(targ, pred)
    dis = (dis < cutoff).nonzero()
    dis = dis[:, (1, 0)]
    unique, idx, counts = torch.unique(dis[...,1], sorted=True, return_inverse=True, return_counts=True)
    ind_sorted = torch.argsort(idx, stable=True)
    cum_sum = counts.cumsum(0)
    cum_sum = torch.cat((torch.tensor([0]), cum_sum[:-1]))
    first_indicies = ind_sorted[cum_sum]
    dis = dis[first_indicies]
    return dis

import torch
b = torch.rand(1000, 3)
c = torch.rand(100, 3)
t = time.time()
for i in range(1000):
    reg = argmatch(b, c, 0.1)
print(time.time() - t)
reg = argmatch(b, c, 0.1).T
print(reg)
print(reg[0].unique().shape, reg[0].shape)


In [ ]:

import torch
a = torch.tensor([1.0])
b = a.to("mps")

In [ ]:
a = [1, 2, 3]
b = [1]
a + b

In [ ]:
import numpy as np
a = np.zeros((10,10))
b = []
b.extend(a)

import torch
# 0.0, 0.5, 1.0; 1.0, 2.0, 3.0
# 0.0, 0.5, 1.0; 0.0, 1.0, 2.0
a = torch.linspace(0, 1, 3)[None, None, :].repeat(1, 3, 1)
a[:, 1] += 1
b = torch.linspace(1, 3, 3)[None, None, :].repeat(1, 3, 1)
c.arccos_().div_(torch.pi).mul_(180)
print(c)

In [ ]:
import multiprocessing as mp
import time

def f():
    print("Zzz")
    time.sleep(5)

p = mp.Pool(6)

_ = [p.apply_async(f) for i in range(12)]

In [ ]:
a = np.zeros((3,3))
a[0] = 1
b = np.random.rand(3,3)
print(a)
print(b)
print(np.matmul(a,b))

In [ ]:
from dataset import AFMGenDataset
from utils import functional
import torch
a  = AFMGenDataset("/Users/supercgor/Documents/data/ice_8_4A/ice_8_4A_small_train.hdf5")
filename ,inp, targ, emb = a[0]
conf, pos, rot = functional.box2orgvec(targ, 0.5, 1.0, (25.0,25.0, 4.0), sort = False, nms = False)



In [ ]:
import torch
import numpy as np
pos = torch.tensor([[23.5279, 19.8613,  0.7911]])
rot = torch.tensor([[[ 0.1403,  0.2997, -0.9437],
        [-0.5516,  0.8152,  0.1769],
        [ 0.8223,  0.4957,  0.2797]]])
from utils.functional import getWaterRotate, makewater
print(pos.shape)
print(makewater(pos,rot))
initdir = np.array([
        [ 0.         , 0.         , 0.9572    ],
        [ 0.9266272  , 0.         ,-0.23998721],
        [ 0.         , 0.88696756 , 0.        ]
        ])
invref = np.linalg.inv(initdir)
pos = np.array([[23.52787009, 19.86131286,  4.7911    ],
 [24.31487009, 20.33571286,  5.0589    ],
 [23.46057009, 20.02011286,  3.8495    ]])
print(pos-pos[0])
getWaterRotate(pos, invref)



In [ ]:
from dataset import AFMGenDataset
from utils import functional
import torch
a  = AFMGenDataset("/Users/supercgor/Documents/data/ice_8_4A/ice_8_4A_small_train.hdf5")
filename ,inp, targ, emb = a[0]

targ2 = functional.box2box(targ)

torch.allclose(targ, targ2, rtol = 1e-4)

In [ ]:
a = torch.ones_like(inp, (1,2,3))

In [ ]:
a  = [1, 2, 3]
b = list(map(lambda x: x + 1, a))
print(a, b)


In [ ]:
import torch
from model import VAELatEmb
a = VAELatEmb()
b = torch.randn((2, 10, 4, 25, 25))
c = torch.randn((2, 1))
out, mu, nu = a(b, c)
out.shape

In [ ]:
import torch
a = torch.randn((2, 4, 4))
torch.where(a[...,(0,)]> 0.5, torch.zeros_like(a), a)

In [ ]:
from dataset import ZVarAFM
from utils.library import encodeWater
import numpy as np
import matplotlib.pyplot as plt

path = "/Users/supercgor/Documents/data/ice_16A/ice_16A_train.hdf5"

dts = ZVarAFM(path, (25, 25, 4))
file_name, inpbox, outbox, emb = dts[0]

ind = inpbox[...,0] == 1

fea = inpbox[ind.nonzero(as_tuple=True)][:,1:]
fea[:,:3] += ind.nonzero()
make = encodeWater(fea)

fig = plt.figure()
ax = fig.add_subplot(121, projection='3d')

ax.title.set_text('inpbox')
# 取出矩陣的每一列的數據
ax.scatter(make[...,0], make[...,1], make[...,2], c='r', marker='o')
ax.scatter(make[...,3], make[...,4], make[...,5], c='b', marker='o')
ax.scatter(make[...,6], make[...,7], make[...,8], c='b', marker='o')
ax.set_aspect('equal', adjustable='box')

ind = outbox[...,0] == 1

fea = outbox[ind.nonzero(as_tuple=True)][:,1:]
fea[:,:3] += ind.nonzero()
make = encodeWater(fea)

ax = fig.add_subplot(122, projection='3d')

ax.title.set_text('outbox')
# 取出矩陣的每一列的數據
ax.scatter(make[...,0], make[...,1], make[...,2], c='r', marker='o')
ax.scatter(make[...,3], make[...,4], make[...,5], c='b', marker='o')
ax.scatter(make[...,6], make[...,7], make[...,8], c='b', marker='o')

ax.set_aspect('equal', adjustable='box')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')


plt.show()

In [ ]:
from dataset import ZVarAFM
from model import VAELatEmb_v2
from utils.library import encodeWater
from utils.model import model_structure
path = "/Users/supercgor/Documents/data/ice_16A/ice_16A_train.hdf5"

dts = ZVarAFM(path, (25, 25, 4))
file_name, inpbox, outbox, emb = dts[0]

inpbox = inpbox[None,...].permute(0, 4, 3, 1, 2)
emb = emb[None, ...]
net = VAELatEmb_v2()

out = net(inpbox, emb)
print("\n".join(model_structure(net)))
out[0].shape

In [ ]:
def f(x):
    return np.clip(x-6, 0, None)%4/10 + np.clip(x,0,6)/10

import numpy as np
a = np.linspace(0, 30, 1000)
b = f(a)

import matplotlib.pyplot as plt
plt.plot(a, b)


In [ ]:
from model import unet_water
import torch
from utils.model import model_structure
net = unet_water(num_res_blocks=(3,3))
a = torch.rand((1, 1, 10, 100, 100))
b = torch.rand((1, 1))
net(a,b).shape
model_structure(net)

In [ ]:
import numpy as np
a = np.array([100, 100, 10], dtype=np.float32)
np.ceil(a/2, out= a)
print(a)
np.ceil(a/2, out= a)
print(a)
np.ceil(a/2, out= a)
print(a)

In [ ]:
for j in range(10):
    pass
print(j)

In [ ]:
def group_as_water(pos_o, pos_h):
    # N, 3 2N, 3 -> N, 9
    dis = torch.cdist(pos_o, pos_h)
    dis = torch.topk(dis, 2, dim = 1, largest = False).indices
    return torch.cat([pos_o,pos_h[dis].view(-1, 6)], dim = -1)
    
from utils import poscar, library
import numpy as np
import torch

card = poscar.load("/Users/supercgor/Library/CloudStorage/OneDrive-北京大学/PKU/Data/bulkice/label/T160_14805.poscar")
real = np.diag(card['lattice'])
dis = card['pos'] * real
print(card.keys())
split_ = card['ion_num']
O, H = np.split(dis, np.cumsum(split_)[:-1], axis=0)
print(O.shape, H.shape)
water = group_as_water(torch.from_numpy(O), torch.from_numpy(H))
code_water = library.decodeWater(water)
print(code_water[1], water[1])

In [ ]:
from utils.library import encodeWater
import numpy as np
import torch
import matplotlib.pyplot as plt
import h5py
path = "/Users/supercgor/Documents/data/union-water-data_train.hdf5"

with h5py.File(path, "r") as f:
    make = f['basal_T160_03_7_1_0']['pos'][()]
    
    
fig = plt.figure()
ax = fig.add_subplot(121, projection='3d')

ax.title.set_text('inpbox')
# 取出矩陣的每一列的數據
ax.scatter(make[...,0], make[...,1], make[...,2], c='r', marker='o')
ax.scatter(make[...,3], make[...,4], make[...,5], c='b', marker='o')
ax.scatter(make[...,6], make[...,7], make[...,8], c='b', marker='o')
ax.set_aspect('equal', adjustable='box')


In [ ]:
a = 1.999
a % 1

In [ ]:
import torch
from model.ndisc import n_layer_disc
from utils import model_structure
net = n_layer_disc(1, 32, [8, 4, 2, 1], [1, 2])
print(net)
image = torch.randn(1, 1, 10, 100, 100)
print(net(image).shape)

print("\n".join(model_structure(net)))


In [ ]:
import torch
from torch import nn
def knn_point(nsample, xyz, new_xyz):
    """
    Input:
        nsample: max sample number in local region
        xyz: all points, [B, N, C]
        new_xyz: query points, [B, S, C]
    Return:
        group_idx: grouped points index, [B, S, nsample]
    """
    sqrdists = square_distance(new_xyz, xyz)
    _, group_idx = torch.topk(sqrdists, nsample, dim = -1, largest=False, sorted=False)
    return group_idx

def square_distance(src, dst):
    """
    Calculate Euclid distance between each two points.
    src^T * dst = xn * xm + yn * ym + zn * zm
    sum(src^2, dim=-1) = xn*xn + yn*yn + zn*zn;
    sum(dst^2, dim=-1) = xm*xm + ym*ym + zm*zm;
    dist = (xn - xm)^2 + (yn - ym)^2 + (zn - zm)^2
         = sum(src**2,dim=-1)+sum(dst**2,dim=-1)-2*src^T*dst
    Input:
        src: source points, [B, N, C]
        dst: target points, [B, M, C]
    Output:
        dist: per-point square distance, [B, N, M]
    """
    B, N, _ = src.shape
    _, M, _ = dst.shape
    dist = -2 * torch.matmul(src, dst.permute(0, 2, 1))
    dist += torch.sum(src ** 2, -1).view(B, N, 1)
    dist += torch.sum(dst ** 2, -1).view(B, 1, M)
    return dist  

def get_graph_feature(coor_q, x_q, coor_k, x_k):

        # coor: bs, 3, np, x: bs, c, np

        k = 16
        batch_size = x_k.size(0)
        num_points_k = x_k.size(2)
        num_points_q = x_q.size(2)

        with torch.no_grad():
#             _, idx = knn(coor_k, coor_q)  # bs k np
            idx = knn_point(k, coor_k.transpose(-1, -2).contiguous(), coor_q.transpose(-1, -2).contiguous()) # B G M
            idx = idx.transpose(-1, -2).contiguous()
            assert idx.shape[1] == k
            idx_base = torch.arange(0, batch_size, device=x_q.device).view(-1, 1, 1) * num_points_k
            idx = idx + idx_base
            idx = idx.view(-1)
        num_dims = x_k.size(1)
        x_k = x_k.transpose(2, 1).contiguous()
        feature = x_k.view(batch_size * num_points_k, -1)[idx, :]
        feature = feature.view(batch_size, k, num_points_q, num_dims).permute(0, 3, 2, 1).contiguous()
        x_q = x_q.view(batch_size, num_dims, num_points_q, 1).expand(-1, -1, -1, k)
        feature = torch.cat((feature - x_q, x_q), dim=1)
        return feature

l1 = nn.Sequential(nn.Conv2d(16, 32, kernel_size=1, bias=False),
                                   nn.GroupNorm(4, 32),
                                   nn.LeakyReLU(negative_slope=0.2)
                                   )

coor = torch.rand(10, 3, 100)
fea = torch.rand(10, 8, 100)
coor2 = torch.rand(10, 3, 70)
fea2 = torch.rand(10, 8, 70)
fea  = get_graph_feature(coor, fea, coor2, fea2)
print(fea.shape)
l1(fea)

In [32]:
import torch
import torch.nn as nn
import numpy as np
from pytorch3d.ops import sample_farthest_points, knn_points, knn_gather, ball_query

def knn_point(nsample, xyz, new_xyz):
    """
    Input:
        nsample: max sample number in local region
        xyz: all points, [B, N, C]
        new_xyz: query points, [B, S, C]
    Return:
        group_idx: grouped points index, [B, S, nsample]
    """
    sqrdists = square_distance(new_xyz, xyz)
    _, group_idx = torch.topk(sqrdists, nsample, dim = -1, largest=False, sorted=False)
    return group_idx

def square_distance(src, dst):
    """
    Calculate Euclid distance between each two points.
    src^T * dst = xn * xm + yn * ym + zn * zm
    sum(src^2, dim=-1) = xn*xn + yn*yn + zn*zn;
    sum(dst^2, dim=-1) = xm*xm + ym*ym + zm*zm;
    dist = (xn - xm)^2 + (yn - ym)^2 + (zn - zm)^2
         = sum(src**2,dim=-1)+sum(dst**2,dim=-1)-2*src^T*dst
    Input:
        src: source points, [B, N, C]
        dst: target points, [B, M, C]
    Output:
        dist: per-point square distance, [B, N, M]
    """
    B, N, _ = src.shape
    _, M, _ = dst.shape
    dist = -2 * torch.matmul(src, dst.permute(0, 2, 1))
    dist += torch.sum(src ** 2, -1).view(B, N, 1)
    dist += torch.sum(dst ** 2, -1).view(B, 1, M)
    return dist  


class DGCNN_Grouper(nn.Module):
    def __init__(self):
        super().__init__()
        '''
        K has to be 16
        '''
        self.input_trans = nn.Conv1d(3, 8, 1)

        self.layer1 = nn.Sequential(nn.Conv2d(16, 32, kernel_size=1, bias=False),
                                   nn.GroupNorm(4, 32),
                                   nn.LeakyReLU(negative_slope=0.2)
                                   )

        self.layer2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1, bias=False),
                                   nn.GroupNorm(4, 64),
                                   nn.LeakyReLU(negative_slope=0.2)
                                   )

        self.layer3 = nn.Sequential(nn.Conv2d(128, 64, kernel_size=1, bias=False),
                                   nn.GroupNorm(4, 64),
                                   nn.LeakyReLU(negative_slope=0.2)
                                   )

        self.layer4 = nn.Sequential(nn.Conv2d(128, 128, kernel_size=1, bias=False),
                                   nn.GroupNorm(4, 128),
                                   nn.LeakyReLU(negative_slope=0.2)
                                   )

    
    @staticmethod
    def fps_downsample(coor, x, num_group):
        xyz = coor.transpose(1, 2).contiguous() # b, n, 3
        print(xyz.shape, num_group)
        fps_points, fps_idx = sample_farthest_points(xyz, K = num_group)
        BQ = ball_query(fps_points, xyz, radius=0.01, K = num_group, return_nn = False)
        
        ball_features, ball_idx = BQ.dists, BQ.idx
        print(ball_features.shape)
        
    @staticmethod
    def get_graph_feature(coor_q, x_q, coor_k, x_k):

        # coor: bs, 3, np, x: bs, c, np

        k = 16
        batch_size = x_k.size(0)
        num_points_k = x_k.size(2)
        num_points_q = x_q.size(2)

        with torch.no_grad():
#             _, idx = knn(coor_k, coor_q)  # bs k np
            idx = knn_points(coor_k.transpose(-1, -2).contiguous(), coor_q.transpose(-1, -2).contiguous(), K = k) # B G M
            idx = idx.idx.transpose(-1, -2).contiguous()
            assert idx.shape[1] == k
            idx_base = torch.arange(0, batch_size, device=x_q.device).view(-1, 1, 1) * num_points_k
            idx = idx + idx_base
            idx = idx.view(-1)
        num_dims = x_k.size(1)
        x_k = x_k.transpose(2, 1).contiguous()
        feature = x_k.view(batch_size * num_points_k, -1)[idx, :]
        feature = feature.view(batch_size, k, num_points_q, num_dims).permute(0, 3, 2, 1).contiguous()
        x_q = x_q.view(batch_size, num_dims, num_points_q, 1).expand(-1, -1, -1, k)
        feature = torch.cat((feature - x_q, x_q), dim=1)
        return feature

    def forward(self, x):

        # x: bs, 3, np

        # bs 3 N(128)   bs C(224)128 N(128)
        coor = x
        f = self.input_trans(x)

        f = self.get_graph_feature(coor, f, coor, f)
        f = self.layer1(f)
        f = f.max(dim=-1, keepdim=False)[0]

        coor_q, f_q = self.fps_downsample(coor, f, 512)
        f = self.get_graph_feature(coor_q, f_q, coor, f)
        f = self.layer2(f)
        f = f.max(dim=-1, keepdim=False)[0]
        coor = coor_q

        f = self.get_graph_feature(coor, f, coor, f)
        f = self.layer3(f)
        f = f.max(dim=-1, keepdim=False)[0]

        coor_q, f_q = self.fps_downsample(coor, f, 128)
        f = self.get_graph_feature(coor_q, f_q, coor, f)
        f = self.layer4(f)
        f = f.max(dim=-1, keepdim=False)[0]
        coor = coor_q

        return coor, f
    
net = DGCNN_Grouper()
a = torch.rand((10, 3, 2048))
net(a)

torch.Size([10, 2048, 3]) 512
torch.Size([10, 512, 512])


TypeError: cannot unpack non-iterable NoneType object

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from pytorch3d.ops import sample_farthest_points, knn_points, knn_gather
# from knn_cuda import KNN
# knn = KNN(k=8, transpose_mode=False)

def knn_point(nsample, xyz, new_xyz):
    """
    Input:
        nsample: max sample number in local region
        xyz: all points, [B, N, C]
        new_xyz: query points, [B, S, C]
    Return:
        group_idx: grouped points index, [B, S, nsample]
    """
    sqrdists = square_distance(new_xyz, xyz)
    _, group_idx = torch.topk(sqrdists, nsample, dim = -1, largest=False, sorted=False)
    return group_idx

def square_distance(src, dst):
    """
    Calculate Euclid distance between each two points.
    src^T * dst = xn * xm + yn * ym + zn * zm;
    sum(src^2, dim=-1) = xn*xn + yn*yn + zn*zn;
    sum(dst^2, dim=-1) = xm*xm + ym*ym + zm*zm;
    dist = (xn - xm)^2 + (yn - ym)^2 + (zn - zm)^2
         = sum(src**2,dim=-1)+sum(dst**2,dim=-1)-2*src^T*dst
    Input:
        src: source points, [B, N, C]
        dst: target points, [B, M, C]
    Output:
        dist: per-point square distance, [B, N, M]
    """
    B, N, _ = src.shape
    _, M, _ = dst.shape
    dist = -2 * torch.matmul(src, dst.permute(0, 2, 1))
    dist += torch.sum(src ** 2, -1).view(B, N, 1)
    dist += torch.sum(dst ** 2, -1).view(B, 1, M)
    return dist   

def get_knn_index(coor_q, coor_k=None):
    coor_k = coor_k if coor_k is not None else coor_q
    # coor: bs, 3, np
    batch_size, _, num_points = coor_q.size()
    num_points_k = coor_k.size(2)

    with torch.no_grad():
#         _, idx = knn(coor_k, coor_q)  # bs k np
        idx = knn_point(8, coor_k.transpose(-1, -2).contiguous(), coor_q.transpose(-1, -2).contiguous()) # B G M
        idx = idx.transpose(-1, -2).contiguous()
        idx_base = torch.arange(0, batch_size, device=coor_q.device).view(-1, 1, 1) * num_points_k
        idx = idx + idx_base
        idx = idx.view(-1)
    
    return idx  # bs*k*np

def get_graph_feature(x, knn_index, x_q=None):

        #x: bs, np, c, knn_index: bs*k*np
        k = 8
        batch_size, num_points, num_dims = x.size()
        num_query = x_q.size(1) if x_q is not None else num_points
        feature = x.view(batch_size * num_points, num_dims)[knn_index, :]
        feature = feature.view(batch_size, k, num_query, num_dims)
        x = x_q if x_q is not None else x
        x = x.view(batch_size, 1, num_query, num_dims).expand(-1, k, -1, -1)
        feature = torch.cat((feature - x, x), dim=-1)
        return feature  # b k np c

class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x


class Attention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        # NOTE scale factor was wrong in my original version, can set manually to be compat with prev weights
        self.scale = qk_scale or head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]   # make torchscript happy (cannot use tensor as tuple)

        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x



class CrossAttention(nn.Module):
    def __init__(self, dim, out_dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        self.dim = dim
        self.out_dim = out_dim
        head_dim = out_dim // num_heads
        # NOTE scale factor was wrong in my original version, can set manually to be compat with prev weights
        self.scale = qk_scale or head_dim ** -0.5

        self.q_map = nn.Linear(dim, out_dim, bias=qkv_bias)
        self.k_map = nn.Linear(dim, out_dim, bias=qkv_bias)
        self.v_map = nn.Linear(dim, out_dim, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)

        self.proj = nn.Linear(out_dim, out_dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, q, v):
        B, N, _ = q.shape
        C = self.out_dim
        k = v
        NK = k.size(1)

        q = self.q_map(q).view(B, N, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3)
        k = self.k_map(k).view(B, NK, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3)
        v = self.v_map(v).view(B, NK, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3)

        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x


class DecoderBlock(nn.Module):
    def __init__(self, dim, num_heads, dim_q = None, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0., attn_drop=0.,
                 drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.self_attn = Attention(
            dim, num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop)
        dim_q = dim_q or dim
        self.norm_q = norm_layer(dim_q)
        self.norm_v = norm_layer(dim)
        self.attn = CrossAttention(
            dim, dim, num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop)
        # NOTE: drop path for stochastic depth, we shall see if this is better than dropout here
        self.drop_path = nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

        self.knn_map = nn.Sequential(
            nn.Linear(dim * 2, dim),
            nn.LeakyReLU(negative_slope=0.2)
        )

        self.merge_map = nn.Linear(dim*2, dim)

        self.knn_map_cross = nn.Sequential(
            nn.Linear(dim * 2, dim),
            nn.LeakyReLU(negative_slope=0.2)
        )

        self.merge_map_cross = nn.Linear(dim*2, dim)

    def forward(self, q, v, self_knn_index=None, cross_knn_index=None):
        # q = q + self.drop_path(self.self_attn(self.norm1(q)))
        norm_q = self.norm1(q)
        q_1 = self.self_attn(norm_q)

        if self_knn_index is not None:
            knn_f = get_graph_feature(norm_q, self_knn_index)
            knn_f = self.knn_map(knn_f)
            knn_f = knn_f.max(dim=1, keepdim=False)[0]
            q_1 = torch.cat([q_1, knn_f], dim=-1)
            q_1 = self.merge_map(q_1)
        
        q = q + self.drop_path(q_1)

        norm_q = self.norm_q(q)
        norm_v = self.norm_v(v)
        q_2 = self.attn(norm_q, norm_v)

        if cross_knn_index is not None:
            knn_f = get_graph_feature(norm_v, cross_knn_index, norm_q)
            knn_f = self.knn_map_cross(knn_f)
            knn_f = knn_f.max(dim=1, keepdim=False)[0]
            q_2 = torch.cat([q_2, knn_f], dim=-1)
            q_2 = self.merge_map_cross(q_2)

        q = q + self.drop_path(q_2)

        # q = q + self.drop_path(self.attn(self.norm_q(q), self.norm_v(v)))
        q = q + self.drop_path(self.mlp(self.norm2(q)))
        return q


class Block(nn.Module):

    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0., attn_drop=0.,
                 drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.attn = Attention(
            dim, num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop)
        # NOTE: drop path for stochastic depth, we shall see if this is better than dropout here
        self.drop_path = nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)

        self.knn_map = nn.Sequential(
            nn.Linear(dim * 2, dim),
            nn.LeakyReLU(negative_slope=0.2)
        )

        self.merge_map = nn.Linear(dim*2, dim)

        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

    def forward(self, x, knn_index = None):
        # x = x + self.drop_path(self.attn(self.norm1(x)))
        norm_x = self.norm1(x)
        x_1 = self.attn(norm_x)

        if knn_index is not None:
            knn_f = get_graph_feature(norm_x, knn_index)
            knn_f = self.knn_map(knn_f)
            knn_f = knn_f.max(dim=1, keepdim=False)[0]
            x_1 = torch.cat([x_1, knn_f], dim=-1)
            x_1 = self.merge_map(x_1)
        
        x = x + self.drop_path(x_1)
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x



class PCTransformer(nn.Module):
    """ Vision Transformer with support for point cloud completion
    """
    def __init__(self, in_chans=3, embed_dim=768, depth=[6, 6], num_heads=6, mlp_ratio=2., qkv_bias=False, qk_scale=None, drop_rate=0., attn_drop_rate=0.,
                        num_query = 224, knn_layer = -1):
        super().__init__()

        self.num_features = self.embed_dim = embed_dim
        
        self.knn_layer = knn_layer

        self.grouper = DGCNN_Grouper()  # B 3 N to B C(3) N(128) and B C(128) N(128)

        self.pos_embed = nn.Sequential(
            nn.Conv1d(in_chans, 128, 1),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(negative_slope=0.2),
            nn.Conv1d(128, embed_dim, 1)
        )

        self.input_proj = nn.Sequential(
            nn.Conv1d(128, embed_dim, 1),
            nn.BatchNorm1d(embed_dim),
            nn.LeakyReLU(negative_slope=0.2),
            nn.Conv1d(embed_dim, embed_dim, 1)
        )

        self.encoder = nn.ModuleList([
            Block(
                dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                drop=drop_rate, attn_drop=attn_drop_rate)
            for i in range(depth[0])])

        # self.increase_dim = nn.Sequential(
        #     nn.Linear(embed_dim,1024),
        #     nn.ReLU(inplace=True),
        #     nn.Linear(1024, 1024)
        # )

        self.increase_dim = nn.Sequential(
            nn.Conv1d(embed_dim, 1024, 1),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(negative_slope=0.2),
            nn.Conv1d(1024, 1024, 1)
        )

        self.num_query = num_query
        self.coarse_pred = nn.Sequential(
            nn.Linear(1024, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 3 * num_query)
        )
        self.mlp_query = nn.Sequential(
            nn.Conv1d(1024 + 3, 1024, 1),
            # nn.BatchNorm1d(1024),
            nn.LeakyReLU(negative_slope=0.2),
            nn.Conv1d(1024, 1024, 1),
            # nn.BatchNorm1d(1024),
            nn.LeakyReLU(negative_slope=0.2),
            nn.Conv1d(1024, embed_dim, 1)
        )

        self.decoder = nn.ModuleList([
            DecoderBlock(
                dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                drop=drop_rate, attn_drop=attn_drop_rate)
            for i in range(depth[1])])


    def pos_encoding_sin_wave(self, coor):
        # ref to https://arxiv.org/pdf/2003.08934v2.pdf
        D = 64 #
        # normal the coor into [-1, 1], batch wise
        normal_coor = 2 * ((coor - coor.min()) / (coor.max() - coor.min())) - 1 

        # define sin wave freq
        freqs = torch.arange(D, dtype=torch.float).cuda() 
        freqs = np.pi * (2**freqs)       

        freqs = freqs.view(*[1]*len(normal_coor.shape), -1) # 1 x 1 x 1 x D
        normal_coor = normal_coor.unsqueeze(-1) # B x 3 x N x 1
        k = normal_coor * freqs # B x 3 x N x D
        s = torch.sin(k) # B x 3 x N x D
        c = torch.cos(k) # B x 3 x N x D
        x = torch.cat([s,c], -1) # B x 3 x N x 2D
        pos = x.transpose(-1,-2).reshape(coor.shape[0], -1, coor.shape[-1]) # B 6D N
        # zero_pad = torch.zeros(x.size(0), 2, x.size(-1)).cuda()
        # pos = torch.cat([x, zero_pad], dim = 1)
        # pos = self.pos_embed_wave(x)
        return pos

    def forward(self, inpc):
        '''
            inpc : input incomplete point cloud with shape B N(2048) C(3)
        '''
        # build point proxy
        bs = inpc.size(0)
        coor, f = self.grouper(inpc.transpose(1,2).contiguous()) 
        knn_index = get_knn_index(coor)
        # NOTE: try to use a sin wave  coor B 3 N, change the pos_embed input dim
        # pos = self.pos_encoding_sin_wave(coor).transpose(1,2)
        pos =  self.pos_embed(coor).transpose(1,2)
        x = self.input_proj(f).transpose(1,2)
        # cls_pos = self.cls_pos.expand(bs, -1, -1)
        # cls_token = self.cls_pos.expand(bs, -1, -1)
        # x = torch.cat([cls_token, x], dim=1)
        # pos = torch.cat([cls_pos, pos], dim=1)
        # encoder
        for i, blk in enumerate(self.encoder):
            if i < self.knn_layer:
                x = blk(x + pos, knn_index)   # B N C
            else:
                x = blk(x + pos)
        # build the query feature for decoder
        # global_feature  = x[:, 0] # B C

        global_feature = self.increase_dim(x.transpose(1,2)) # B 1024 N 
        global_feature = torch.max(global_feature, dim=-1)[0] # B 1024

        coarse_point_cloud = self.coarse_pred(global_feature).reshape(bs, -1, 3)  #  B M C(3)

        new_knn_index = get_knn_index(coarse_point_cloud.transpose(1, 2).contiguous())
        cross_knn_index = get_knn_index(coor_k=coor, coor_q=coarse_point_cloud.transpose(1, 2).contiguous())

        query_feature = torch.cat([
            global_feature.unsqueeze(1).expand(-1, self.num_query, -1), 
            coarse_point_cloud], dim=-1) # B M C+3 
        q = self.mlp_query(query_feature.transpose(1,2)).transpose(1,2) # B M C 
        # decoder
        for i, blk in enumerate(self.decoder):
            if i < self.knn_layer:
                q = blk(q, x, new_knn_index, cross_knn_index)   # B M C
            else:
                q = blk(q, x)

        return q, coarse_point_cloud
    
net = PCTransformer()
a = torch.rand(10, 128, 3)
net(a)


In [ ]:
import torch
import pytorch3d
a = torch.randn(10, 100, 3)
pytorch3d.ops.sample_farthest_points(a)

In [57]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from time import time
import numpy as np
from pytorch3d.ops import sample_farthest_points, knn_points, knn_gather, ball_query


def timeit(tag, t):
    print("{}: {}s".format(tag, time() - t))
    return time()

def pc_normalize(pc):
    l = pc.shape[0]
    centroid = np.mean(pc, axis=0)
    pc = pc - centroid
    m = np.max(np.sqrt(np.sum(pc**2, axis=1)))
    pc = pc / m
    return pc

def square_distance(src, dst):
    """
    Calculate Euclid distance between each two points.

    src^T * dst = xn * xm + yn * ym + zn * zm；
    sum(src^2, dim=-1) = xn*xn + yn*yn + zn*zn;
    sum(dst^2, dim=-1) = xm*xm + ym*ym + zm*zm;
    dist = (xn - xm)^2 + (yn - ym)^2 + (zn - zm)^2
         = sum(src**2,dim=-1)+sum(dst**2,dim=-1)-2*src^T*dst

    Input:
        src: source points, [B, N, C]
        dst: target points, [B, M, C]
    Output:
        dist: per-point square distance, [B, N, M]
    """
    B, N, _ = src.shape
    _, M, _ = dst.shape
    dist = -2 * torch.matmul(src, dst.permute(0, 2, 1))
    dist += torch.sum(src ** 2, -1).view(B, N, 1)
    dist += torch.sum(dst ** 2, -1).view(B, 1, M)
    return dist


def index_points(points, idx):
    """

    Input:
        points: input points data, [B, N, C]
        idx: sample index data, [B, S]
    Return:
        new_points:, indexed points data, [B, S, C]
    """
    device = points.device
    B = points.shape[0]
    view_shape = list(idx.shape)
    view_shape[1:] = [1] * (len(view_shape) - 1)
    repeat_shape = list(idx.shape)
    repeat_shape[0] = 1
    batch_indices = torch.arange(B, dtype=torch.long).to(device).view(view_shape).repeat(repeat_shape)
    new_points = points[batch_indices, idx, :]
    return new_points


def farthest_point_sample(xyz, npoint):
    """
    Input:
        xyz: pointcloud data, [B, N, 3]
        npoint: number of samples
    Return:
        centroids: sampled pointcloud index, [B, npoint]
    """
    centroids = sample_farthest_points(xyz, K = npoint)[1]
    return centroids


def query_ball_point(radius, nsample, xyz, new_xyz):
    """
    Input:
        radius: local region radius
        nsample: max sample number in local region
        xyz: all points, [B, N, 3]
        new_xyz: query points, [B, S, 3]
    Return:
        group_idx: grouped points index, [B, S, nsample]
    """
    out = ball_query(new_xyz, xyz, K = nsample, radius = radius).idx
    return out


def sample_and_group(npoint, radius, nsample, xyz, points, returnfps=False):
    """
    Input:
        npoint:
        radius:
        nsample:
        xyz: input points position data, [B, N, 3]
        points: input points data, [B, N, D]
    Return:
        new_xyz: sampled points position data, [B, npoint, nsample, 3]
        new_points: sampled points data, [B, npoint, nsample, 3+D]
    """
    B, N, C = xyz.shape
    S = npoint
    fps_idx = farthest_point_sample(xyz, npoint) # [B, npoint, C]
    new_xyz = index_points(xyz, fps_idx)
    idx = query_ball_point(radius, nsample, xyz, new_xyz)
    grouped_xyz = index_points(xyz, idx) # [B, npoint, nsample, C]
    grouped_xyz_norm = grouped_xyz - new_xyz.view(B, S, 1, C)

    if points is not None:
        grouped_points = index_points(points, idx)
        new_points = torch.cat([grouped_xyz_norm, grouped_points], dim=-1) # [B, npoint, nsample, C+D]
    else:
        new_points = grouped_xyz_norm
    if returnfps:
        return new_xyz, new_points, grouped_xyz, fps_idx
    else:
        return new_xyz, new_points


def sample_and_group_all(xyz, points):
    """
    Input:
        xyz: input points position data, [B, N, 3]
        points: input points data, [B, N, D]
    Return:
        new_xyz: sampled points position data, [B, 1, 3]
        new_points: sampled points data, [B, 1, N, 3+D]
    """
    device = xyz.device
    B, N, C = xyz.shape
    new_xyz = torch.zeros(B, 1, C).to(device)
    grouped_xyz = xyz.view(B, 1, N, C)
    if points is not None:
        new_points = torch.cat([grouped_xyz, points.view(B, 1, N, -1)], dim=-1)
    else:
        new_points = grouped_xyz
    return new_xyz, new_points


class PointNetSetAbstraction(nn.Module):
    def __init__(self, npoint, radius, nsample, in_channel, mlp, group_all):
        super(PointNetSetAbstraction, self).__init__()
        self.npoint = npoint
        self.radius = radius
        self.nsample = nsample
        self.mlp_convs = nn.ModuleList()
        self.mlp_bns = nn.ModuleList()
        last_channel = in_channel
        for out_channel in mlp:
            self.mlp_convs.append(nn.Conv2d(last_channel, out_channel, 1))
            self.mlp_bns.append(nn.BatchNorm2d(out_channel))
            last_channel = out_channel
        self.group_all = group_all

    def forward(self, xyz, points):
        """
        Input:
            xyz: input points position data, [B, C, N]
            points: input points data, [B, D, N]
        Return:
            new_xyz: sampled points position data, [B, C, S]
            new_points_concat: sample points feature data, [B, D', S]
        """
        xyz = xyz.permute(0, 2, 1)
        if points is not None:
            points = points.permute(0, 2, 1)

        if self.group_all:
            new_xyz, new_points = sample_and_group_all(xyz, points)
        else:
            new_xyz, new_points = sample_and_group(self.npoint, self.radius, self.nsample, xyz, points)
        # new_xyz: sampled points position data, [B, npoint, C]
        # new_points: sampled points data, [B, npoint, nsample, C+D]
        new_points = new_points.permute(0, 3, 2, 1) # [B, C+D, nsample,npoint]
        for i, conv in enumerate(self.mlp_convs):
            bn = self.mlp_bns[i]
            new_points =  F.relu(bn(conv(new_points)))

        new_points = torch.max(new_points, 2)[0]
        new_xyz = new_xyz.permute(0, 2, 1)
        return new_xyz, new_points


class PointNetSetAbstractionMsg(nn.Module):
    def __init__(self, npoint, radius_list, nsample_list, in_channel, mlp_list):
        super(PointNetSetAbstractionMsg, self).__init__()
        self.npoint = npoint
        self.radius_list = radius_list
        self.nsample_list = nsample_list
        self.conv_blocks = nn.ModuleList()
        self.bn_blocks = nn.ModuleList()
        for i in range(len(mlp_list)):
            convs = nn.ModuleList()
            bns = nn.ModuleList()
            last_channel = in_channel + 3
            for out_channel in mlp_list[i]:
                convs.append(nn.Conv2d(last_channel, out_channel, 1))
                bns.append(nn.BatchNorm2d(out_channel))
                last_channel = out_channel
            self.conv_blocks.append(convs)
            self.bn_blocks.append(bns)

    def forward(self, xyz, points):
        """
        Input:
            xyz: input points position data, [B, C, N]
            points: input points data, [B, D, N]
        Return:
            new_xyz: sampled points position data, [B, C, S]
            new_points_concat: sample points feature data, [B, D', S]
        """
        xyz = xyz.permute(0, 2, 1)
        if points is not None:
            points = points.permute(0, 2, 1)

        B, N, C = xyz.shape
        S = self.npoint
        new_xyz = index_points(xyz, farthest_point_sample(xyz, S))
        new_points_list = []
        for i, radius in enumerate(self.radius_list):
            K = self.nsample_list[i]
            group_idx = query_ball_point(radius, K, xyz, new_xyz)
            grouped_xyz = index_points(xyz, group_idx)
            grouped_xyz -= new_xyz.view(B, S, 1, C)
            if points is not None:
                grouped_points = index_points(points, group_idx)
                grouped_points = torch.cat([grouped_points, grouped_xyz], dim=-1)
            else:
                grouped_points = grouped_xyz

            grouped_points = grouped_points.permute(0, 3, 2, 1)  # [B, D, K, S]
            for j in range(len(self.conv_blocks[i])):
                conv = self.conv_blocks[i][j]
                bn = self.bn_blocks[i][j]
                grouped_points =  F.relu(bn(conv(grouped_points)))
            new_points = torch.max(grouped_points, 2)[0]  # [B, D', S]
            new_points_list.append(new_points)

        new_xyz = new_xyz.permute(0, 2, 1)
        new_points_concat = torch.cat(new_points_list, dim=1)
        return new_xyz, new_points_concat


class PointNetFeaturePropagation(nn.Module):
    def __init__(self, in_channel, mlp):
        super(PointNetFeaturePropagation, self).__init__()
        self.mlp_convs = nn.ModuleList()
        self.mlp_bns = nn.ModuleList()
        last_channel = in_channel
        for out_channel in mlp:
            self.mlp_convs.append(nn.Conv1d(last_channel, out_channel, 1))
            self.mlp_bns.append(nn.BatchNorm1d(out_channel))
            last_channel = out_channel

    def forward(self, xyz1, xyz2, points1, points2):
        """
        Input:
            xyz1: input points position data, [B, C, N]
            xyz2: sampled input points position data, [B, C, S]
            points1: input points data, [B, D, N]
            points2: input points data, [B, D, S]
        Return:
            new_points: upsampled points data, [B, D', N]
        """
        xyz1 = xyz1.permute(0, 2, 1)
        xyz2 = xyz2.permute(0, 2, 1)

        points2 = points2.permute(0, 2, 1)
        B, N, C = xyz1.shape
        _, S, _ = xyz2.shape

        if S == 1:
            interpolated_points = points2.repeat(1, N, 1)
        else:
            dists = square_distance(xyz1, xyz2)
            dists, idx = dists.sort(dim=-1)
            dists, idx = dists[:, :, :3], idx[:, :, :3]  # [B, N, 3]

            dist_recip = 1.0 / (dists + 1e-8)
            norm = torch.sum(dist_recip, dim=2, keepdim=True)
            weight = dist_recip / norm
            interpolated_points = torch.sum(index_points(points2, idx) * weight.view(B, N, 3, 1), dim=2)

        if points1 is not None:
            points1 = points1.permute(0, 2, 1)
            new_points = torch.cat([points1, interpolated_points], dim=-1)
        else:
            new_points = interpolated_points

        new_points = new_points.permute(0, 2, 1)
        for i, conv in enumerate(self.mlp_convs):
            bn = self.mlp_bns[i]
            new_points = F.relu(bn(conv(new_points)))
        return new_points

In [59]:
import torch.nn as nn
import torch.nn.functional as F

class get_model(nn.Module):
    def __init__(self, num_classes):
        super(get_model, self).__init__()
        self.sa1 = PointNetSetAbstraction(1024, 0.1, 32, 9 + 3, [32, 32, 64], False)
        self.sa2 = PointNetSetAbstraction(256, 0.2, 32, 64 + 3, [64, 64, 128], False)
        self.sa3 = PointNetSetAbstraction(64, 0.4, 32, 128 + 3, [128, 128, 256], False)
        self.sa4 = PointNetSetAbstraction(16, 0.8, 32, 256 + 3, [256, 256, 512], False)
        self.fp4 = PointNetFeaturePropagation(768, [256, 256])
        self.fp3 = PointNetFeaturePropagation(384, [256, 256])
        self.fp2 = PointNetFeaturePropagation(320, [256, 128])
        self.fp1 = PointNetFeaturePropagation(128, [128, 128, 128])
        self.conv1 = nn.Conv1d(128, 128, 1)
        self.bn1 = nn.BatchNorm1d(128)
        self.drop1 = nn.Dropout(0.5)
        self.conv2 = nn.Conv1d(128, num_classes, 1)

    def forward(self, xyz):
        l0_points = xyz
        l0_xyz = xyz[:,:3,:]

        l1_xyz, l1_points = self.sa1(l0_xyz, l0_points)
        l2_xyz, l2_points = self.sa2(l1_xyz, l1_points)
        l3_xyz, l3_points = self.sa3(l2_xyz, l2_points)
        l4_xyz, l4_points = self.sa4(l3_xyz, l3_points)

        l3_points = self.fp4(l3_xyz, l4_xyz, l3_points, l4_points)
        l2_points = self.fp3(l2_xyz, l3_xyz, l2_points, l3_points)
        l1_points = self.fp2(l1_xyz, l2_xyz, l1_points, l2_points)
        l0_points = self.fp1(l0_xyz, l1_xyz, None, l1_points)

        x = self.drop1(F.relu(self.bn1(self.conv1(l0_points))))
        x = self.conv2(x)
        x = F.log_softmax(x, dim=1)
        x = x.permute(0, 2, 1)
        return x, l4_points


class get_loss(nn.Module):
    def __init__(self):
        super(get_loss, self).__init__()
    def forward(self, pred, target, trans_feat, weight):
        total_loss = F.nll_loss(pred, target, weight=weight)

        return total_loss

if __name__ == '__main__':
    import  torch
    model = get_model(13)
    xyz = torch.rand(6, 9, 2048)

In [64]:
x, l4_points = model(xyz)
print(x.shape, l4_points.shape)

torch.Size([6, 2048, 13]) torch.Size([6, 512, 16])


In [69]:
from model.unet_v3 import unet_water

import torch
a = unet_water(embedding_input=0)
b = torch.rand((1, 1, 10, 100, 100))
a(b)

tensor([[[[[-4.6364e-02, -5.2330e-03, -3.2129e-03,  ..., -3.0419e-02,
            -3.8897e-02, -8.3740e-03],
           [-5.9711e-02, -8.8515e-02, -4.9443e-02,  ..., -6.6577e-02,
            -5.0221e-02, -2.6143e-02],
           [-6.4317e-02, -1.1535e-02, -3.9840e-02,  ..., -3.0520e-02,
             3.4140e-03, -3.7609e-02],
           ...,
           [-5.2618e-02, -8.2175e-02, -3.0063e-02,  ..., -6.2769e-02,
            -5.0686e-02, -5.2275e-02],
           [-2.4826e-02, -2.1310e-02, -2.8951e-02,  ..., -4.9462e-02,
            -2.3668e-03, -5.1266e-02],
           [-1.6182e-02, -5.7589e-02, -3.1335e-02,  ..., -7.7288e-02,
            -6.7831e-02, -2.3971e-02]],

          [[-5.9214e-02, -2.7416e-02,  1.4104e-03,  ..., -4.6262e-02,
            -8.2154e-04, -2.7600e-02],
           [-3.6558e-02, -8.0564e-02, -6.8477e-02,  ..., -5.3400e-03,
            -7.1562e-02, -8.6677e-02],
           [-5.3189e-02, -5.6111e-02, -4.9219e-02,  ..., -3.7220e-02,
            -2.9924e-02, -1.0399e-01],
 